### Storing economic data in a SQL database



In [1]:
import requests
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('econ_data.db')

In [19]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
param = [('name', 'Inflation'),
         ('dataset', 'IFS'),
         ('freq', 'M'),
         ('country', 'GB+DE+US+FR+CA+BR+SG+JP'),
         ('series', 'PCPI_IX')]
series = '.'.join([param[x][1] for x in [2,3,4]])

key = 'CompactData/{}/{}'.format(param[1][1], series)

In [20]:
r = requests.get('{}{}'.format(url, key)).json()
df = pd.DataFrame()
for s in r['CompactData']['DataSet']['Series']:
    df[s['@REF_AREA']] = pd.Series([float(x['@OBS_VALUE']) for x in s['Obs']],
                                  index=[x['@TIME_PERIOD'] for x in s['Obs']])

df = df.pct_change(12).multiply(100).astype(float).round(1)

In [21]:
df.to_sql(param[0][1], conn, if_exists='replace', index_label='Date')

In [22]:
pd.read_sql("select * from Inflation", conn, index_col='Date', parse_dates=True).tail()

,JP,CA,GB,BR,SG,DE,US,FR
Date,,,,,,,,
2017-06,0.3,1.0,2.7,3.0,0.6,1.6,1.6,0.7
2017-07,0.5,1.2,2.6,2.7,0.5,1.7,1.7,0.7
2017-08,0.6,1.4,2.9,2.5,0.4,1.8,1.9,0.9
2017-09,0.7,1.6,3.0,2.5,0.3,1.8,2.2,1.0
2017-10,0.2,1.4,3.0,2.7,0.4,1.6,2.0,1.1


In [14]:
df

,JP,CA,GB,BR,SG,DE,US,FR
1957-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1957-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1957-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1957-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1957-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1957-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1957-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1957-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1957-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1957-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
